In [ ]:
# Source data of Local Authority Disticts (LADs): Ordnance Survey Boundary Line https://www.ordnancesurvey.co.uk/opendatadownload/products.html#BDLINE
# Alternative source data: http://geoportal.statistics.gov.uk/datasets/local-authority-districts-december-2016-full-clipped-boundaries-in-great-britain
# Direct link: Shapefile
# Source data from HMCTS: https://jac.judiciary.gov.uk/sites/default/files/sync/application_selection/map-uk-courts-regions-hmcts-march2012.pdf

In [ ]:
import pandas as pd
from simpledbf import Dbf5

dbf = Dbf5('district_borough_hmcts_regions.dbf')
df = dbf.to_dataframe()

In [ ]:
df2 = df[["code", "hmcts"]]

In [ ]:
df2.columns = ["lad16cd_code", "hmcts_region"]

In [ ]:
df2.to_csv("lad_hmcts_region_lookup.csv", encoding="utf-8", index=False)

In [ ]:
%%bash 
shp2pgsql -I -s 27700 lad/Local_Authority_Districts_December_2016_Full_Clipped_Boundaries_in_Great_Britain.shp public.las_ons | psql -d postgres

In [ ]:
%%bash 
shp2pgsql -I -s 27700 bdline/Data/GB/district_borough_unitary_ward_region.shp public.las_os | psql -d postgres

In [ ]:
import psycopg2
con_string = "host='localhost' dbname='postgres' user='robinlinacre' password=''"
conn = psycopg2.connect(con_string)
cursor = conn.cursor()

from sqlalchemy import create_engine
engine = create_engine('postgresql://robinlinacre:@localhost:5432/postgres') 
df2.to_sql("lad_hmcts_region_lookup", engine, index=False, if_exists="replace")

In [ ]:
# How to add ONS or OS shapefiles to Postgres
sql = """
create table hmcts_regions as
select st_union(ST_snaptogrid(geom,0.0001)), hmcts_region
from las_ons as shp
left join lad_hmcts_region_lookup as l
on shp.lad16cd = l.lad16cd_code
group by hmcts_region
"""
cursor.execute(sql)
conn.commit()

In [ ]:
# Write postgres data to shapefile on disk
%%bash
pgsql2shp -f new_shapefiles -h localhost -u robinlinacre postgres "SELECT * from hmcts_regions"

In [ ]:
%%bash
ogr2ogr -t_srs WGS84  -f GeoJSON   fixed.geojson   new_shapefiles.shp -simplify 100
topojson -o fixed.topojson fixed.geojson --simplify-proportion 0.04 -p